## Initial API workflow for DDH2.0

- Add a dataset
- Add resources
- Update existing resources
- Publish the dataset
- Move dataset from Publish to Draft
- Make changes to draft version and re-publish dataset
- View data/metadata versions
- Update existing versions 

In [2]:
import urllib
import requests
import zipfile
from msrestazure.azure_active_directory import AADTokenCredentials

## Other required imports
import adal, uuid, time
from datetime import datetime as dt
import numpy as np
import pandas as pd
import json

import os, sys
sys.path.append("../../API/RomaniaHub/ddh2_testing")
from ddh2 import dataset

sys.path.append("../../API/pyddh")
import ddh
ddh.load('ddh1stg.prod.acquia-sites.com')

In [5]:
with open("sample_params.json") as f:
    sample_parameters = json.load(f)

In [2]:
ddh_config = {}
ddh_config['host'] = "ddh1stg.prod.acquia-sites.com"
ddh_config['protocol'] = 'https'
ddh_config['authorityHostUrl'] = sample_parameters['authorityHostUrl']
ddh_config['clientId'] = sample_parameters['clientId']
ddh_config['tenant'] = sample_parameters['tenant']
ddh_config['resource'] = sample_parameters['resource']

In [7]:
dataset.load(ddh_config)

#### Adding dataset

In [5]:
full_ds = {
    "Dataset": {
        "identification": {
            "title": "Workflow Test QA 1",
            "subtitle": "testing the basic workflow",
            "description": "description for testing basic workflow in QA",
            "acronym": "WDI",
            "topics": [ "Agriculture and Food Security", "Economic Growth" ],
            "practice": { "code": "POV" },
            "wb_project_reference": "P679812",
            "grant_number": "Grant number 01",
            "collection_id": [ "testing 1", "Type 2" ],
            "translated_title": "World Development Indicators TEST ENG",
            "point_of_contact": [
                {
                    "name": "Gaurav Bhardwaj",
                    "role": "OWNER",
                    "email": "gbhardwaj@worldbank.org",
                    "type": "TTL",
                    "upi": "000542830",
                    "is_emailaddress_visibility_externally": 'true'
                },
                {
                    "name": "User 2",
                    "role": "RESOURCE_PROVIDER",
                    "email": "user2@abc.com",
                    "type": "Type2",
                    "upi": "5577071",
                    "is_emailaddress_visibility_externally": 'false'
                }
            ],
            "dates": [
                {
                    "date": "12/01/2020",
                    "type": "Type1"

                },
                {
                    "date": "12/02/2020",
                    "type": "Type2"

                }
            ],
            "language_supported": [
                {
                    "code": "EN"
                },
                {
                    "code": "FR"
                }
            ]
        },
        "keywords": [
            {
                "name": "Tag 1"
            },
            {
                "name": "Key Word 1"
            }
        ],
        "constraints": {
            "security": {
                "classification": "OFFICIAL_USE_ONLY",
                "exception": "WBG exception1",
                "userNote": "WBG userNote1"
            },
            "license": {
                "license_id": "Creative Commons 4.0"
            }
        },
        "data_quality": {
            "data_notes": "WBG data_notes"
        },
        "temporal_extent": {
            "end_date": "5/27/2020 6:30:00 PM",
            "start_date": "5/26/2020 6:30:00 PM",
            "description": "Temporal description",
            "duration": "1 Month",
            "coverage": [ "Coverage1", "Coverage2" ]
        },
        "temporal_resolution": {
            "periodicity": "HOUR_MIN"
        },
        "spatial_resolution": {
            "granularity": "HOUSEHOLD",
            "pixel_resolution": "1024"
        },
        "reference_system": {
            "reference_id": "WBG reference_id",
            "version": "1.0",
            "map_projection": "WBG map_projection"
        },
        "lineage": {
            "source_type": "Government Agency",
            "source": "source",
            "source_reference": "source_reference",
            "statistical_concept_and_methodology": "statistical_concept_and_methodology",
            "base_period": "5/25/2020",
            "aggregation_method": "WBG aggregation_method",
            "description": "WBG description",
            "related_links_and_publications": [ "link1", "link2" ],
            "publication_place": "WBG publication_place",
            "funding_name_abbreviation_role": [ "name1", "name2" ],
            "study_type": "WBG study_type",
            "deviations_from_sample_design": "WBG deviations_from_sample_design",
            "other_acknowledgements": "WBG other acknowledgements"
        },
        "maintenance_information": {
            "update_frequency": "QUARTER",
            "update_schedule": "April, July, September, December",
            "maintenance_note": "WBG maintenance note1",
            "embargo_date": "5/25/2020 6:30:00 PM",
            "embargo_comments": "WBG embargo comments",
            "version_description": "WBG version description1",
            "version_production_date": "5/25/2020",
            "version_notes": "WBG version notes"
        },
        "subscription": {
            "subscription_date_from": "5/28/2020",
            "subscription_date_to": "5/29/2020"
        }
    }
}


In [6]:
req = requests.post("https://ddhinboundapiqa.asestg.worldbank.org/dataset/create",
                   json = full_ds, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiqa.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiqa.asestg.worldbank.org:443 "POST /dataset/create HTTP/1.1" 200 None


In [7]:
req.text

'EntityID: 0c69d0f3-60e8-ea11-a817-000d3a33f89d'

#### Adding a resource

In [11]:
res_json = {
    "Resource": {        
        "dataset_id":"0c69d0f3-60e8-ea11-a817-000d3a33f89d",
        "identification": {
            "name": "Workflow Test - Resource 1",
            "type": "Download",
            "description": "Doc file for Workflow Test 1",
            "status_log": "WBG status_log",
            #"file_name": "wbg.txt",
            "version": "1",
            "version_date": "8/18/2020",
            "citation": "temp citation",
            "tags": [
                {
                    "name": "test_tag"
                },
                {
                    "name": "uat_testing"
                }
            ]
        },
        "constraints": {
            "security": {
                "data_classification_of_file": { "code": "OFFICIAL_USE_ONLY" },
                "exception": "Third party confidence"
            }
            #"license": {
            #    "license_id": "WBG license id",
            #    "custom_license_information": "WBG custom license information",
            #    "license_reference": "WBG license reference"
            #}
        },
        "distribution": {
            "format": "pdf",
            "geospatial_api_format": "WBG geospatial api format",
            "url": "http://aaaa.com/file.pdf",
            "website_url": "http://aaaa.com",
            "distribution_description": "WBG distribution description",
            "distribution_format": "pdf",
            "distribution_size": "100 MB"
        },       
        "temporal_extent": {
            "end_date": "8/26/2020",
            "start_date": "5/24/2020",
            "description": "temporal resolution test",
            "temporal_resolution": "some resolution"
        }
    }
}


In [12]:
res_create = "https://ddhinboundapiqa.asestg.worldbank.org/resource/create"

req_res = requests.post(res_create, json = res_json, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiqa.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiqa.asestg.worldbank.org:443 "POST /resource/create HTTP/1.1" 200 None


In [13]:
req_res.text

'EntityID: 1778b03f-61e8-ea11-a817-000d3a33f89d'

#### Adding resource file

In [ ]:

{
"DatasetUniqueNumber":"0004516",
"DataSetId":"0c69d0f3-60e8-ea11-a817-000d3a33f89d",
"DataResourceUniqueNumber":"DR0006717",
"ResourceId":"1778b03f-61e8-ea11-a817-000d3a33f89d",
"FileName":"WDR2011.csv",
"MimeType":"csv"
}


#### Publish the dataset now

In [14]:
publish_url = "https://ddhinboundapiqa.asestg.worldbank.org/workflow/updatestatus"

publish_js = {
    "id": "0c69d0f3-60e8-ea11-a817-000d3a33f89d",
    "dataset_status": "PUBLISHED",
    "publish_label":"v1 publish"
}

req_pub = requests.post(publish_url, json = publish_js, headers = dataset.ddh_header)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiqa.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiqa.asestg.worldbank.org:443 "POST /workflow/updatestatus HTTP/1.1" 200 None


In [15]:
req_pub.text

'EntityID: 9211da06-62e8-ea11-a817-000d3a5bf333'

#### At this point dataset is published and the files move from staging to prod.

#### Adding another resource

In [14]:
res_json2 = {
    "Resource": {        
        "dataset_id":"f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
        "identification": {
            "name": "Workflow Test - Resource 2",
            "type": "Download",
            "description": "Doc file for Workflow Test 2",
            "status_log": "WBG status_log",
            #"file_name": "wbg.txt",
            "version": "1",
            "version_date": "8/18/2020",
            "citation": "temp citation",
            "tags": [
                {
                    "name": "test_tag"
                },
                {
                    "name": "uat_testing"
                }
            ]
        },
        "constraints": {
            "security": {
                "data_classification_of_file": { "code": "PUBLIC" },
                "exception": "Third party confidence"
            }
            #"license": {
            #    "license_id": "WBG license id",
            #    "custom_license_information": "WBG custom license information",
            #    "license_reference": "WBG license reference"
            #}
        },
        "distribution": {
            #"format": "pdf",
            "geospatial_api_format": "WBG geospatial api format",
            #"url": "http://aaaa.com/file.pdf",
            #"website_url": "http://aaaa.com",
            #"distribution_description": "WBG distribution description",
            #"distribution_format": "pdf",
            #"distribution_size": "100 MB"
        },       
        "temporal_extent": {
            "end_date": "8/26/2020",
            "start_date": "5/24/2020",
            "description": "temporal resolution test",
            "temporal_resolution": "some resolution"
        }
    }
}


In [15]:
res_create = "https://ddhinboundapiuat.asestg.worldbank.org/resource/create"

req_res = requests.post(res_create, json = res_json2, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /resource/create HTTP/1.1" 200 None


In [17]:
req_res.text

'EntityID: a5e17900-f6e2-ea11-a813-000d3a8b33eb'

#### Republihsing the data

In [18]:
publish_url = "https://ddhinboundapiuat.asestg.worldbank.org/workflow/updatestatus"

publish_js = {
    "id": "f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
    "dataset_status": "PUBLISHED",
    "publish_label":"v1 publish"
}

req_pub2 = requests.post(publish_url, json = publish_js, headers = dataset.ddh_header)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /workflow/updatestatus HTTP/1.1" 200 None


In [19]:
req_pub2.text

'EntityID: 87deb2a5-f6e2-ea11-a813-000d3a191e87'

#### Adding data file to second resource

In [ ]:
{
"DatasetUniqueNumber":"0001103",
"DataSetId":"f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
"DataResourceUniqueNumber":"DR0001056",
"ResourceId":"a5e17900-f6e2-ea11-a813-000d3a8b33eb",
"FileName":"DoingBusiness.docx",
"MimeType":"docx"
}

#### Publihsing again with added data file

In [180]:
publish_url = "https://ddhinboundapiuat.asestg.worldbank.org/workflow/updatestatus"

publish_js = {
    "id": "f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
    "dataset_status": "PUBLISHED",
    "publish_label":"v5 publish"
}

req_pub3 = requests.post(publish_url, json = publish_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /workflow/updatestatus HTTP/1.1" 200 None


In [181]:
req_pub3.text

'EntityID: 04cccd50-9ee7-ea11-a817-000d3a191e87'

#### Updating published dataset

In [9]:
geo_js = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              77.1240234375,
              28.8831596093235
            ],
            [
              76.80541992187499,
              28.51696944040106
            ],
            [
              77.332763671875,
              28.256005619824972
            ],
            [
              77.684326171875,
              28.565225490654658
            ],
            [
              77.40966796875,
              28.87353946316266
            ],
            [
              77.1240234375,
              28.8831596093235
            ]
          ]
        ]
      }
    }
  ]
}

In [10]:
up2_2 = {"Dataset" :
{"id" : "f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
 "name": "Workflow Test 1",
"identification" : {"id": "f4ba5f65-efe2-ea11-a813-000d3a8b33eb",
        "title": "Workflow Test 1"},
 "geographical_extent": {"id": "fdba5f65-efe2-ea11-a813-000d3a8b33eb",
            #"geo_coverage": "WBG geo_coverage",
            "countries": [
                {
                    "code": "IN"
                }
            ],
            "bounding_box_geo_json": str(geo_js),
            "bounding_box": {
                "west_longitude": "77.1240234375",
                "east_longitude": "28.256005619824972",
                "north_latitude": "77.684326171875",
                "south_latitude": "28.8831596093235"
            }
        },
}
}

In [6]:
up_poc = {"Dataset" :
{"id" : "0c69d0f3-60e8-ea11-a817-000d3a33f89d",
 #"name": "Workflow Test QA 1",
"identification" : {"id": "0d69d0f3-60e8-ea11-a817-000d3a33f89d",
        "title": "Workflow Test QA 1",
        "grant_number": "Grant number # 8762191"
                   }
}
}

In [7]:
up_poc

{'Dataset': {'id': '0c69d0f3-60e8-ea11-a817-000d3a33f89d',
  'identification': {'id': '0d69d0f3-60e8-ea11-a817-000d3a33f89d',
   'title': 'Workflow Test QA 1',
   'grant_number': 'Grant number # 8762191'}}}

In [14]:
update_url = "https://ddhinboundapiqa.asestg.worldbank.org/dataset/update"

req_up_2 = requests.post(update_url, json = up_poc, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiqa.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiqa.asestg.worldbank.org:443 "POST /dataset/update HTTP/1.1" 200 None


In [15]:
req_up_2.text

'EntityID: 0c69d0f3-60e8-ea11-a817-000d3a33f89d'

#### Moving published dataset to DRAFT

In [75]:
publish_url = "https://ddhinboundapiqa.asestg.worldbank.org/workflow/updatestatus"

publish_js = {
    "id": "0c69d0f3-60e8-ea11-a817-000d3a33f89d",
    "dataset_status": "DRAFT",
    "publish_label": "v2 draft"
}

req_pub4 = requests.post(publish_url, json = publish_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiqa.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiqa.asestg.worldbank.org:443 "POST /workflow/updatestatus HTTP/1.1" 200 None


In [76]:
req_pub4.text

'EntityID: 0c69d0f3-60e8-ea11-a817-000d3a33f89d'

#### Re-trying metadata update

In [31]:
update_url = "https://ddhinboundapiuat.asestg.worldbank.org/dataset/update"

req_up_3 = requests.post(update_url, json = up2_2, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /dataset/update HTTP/1.1" 200 None


In [34]:
req_up_3.text

'EntityID: f3ba5f65-efe2-ea11-a813-000d3a8b33eb'

#### Publihsing updated data

In [168]:
publish_url = "https://ddhinboundapiuat.asestg.worldbank.org/workflow/updatestatus"

publish_js = {
    "id": "f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
    "dataset_status": "PUBLISHED",
    "publish_label": "v9 publish"
}

req_pub5 = requests.post(publish_url, json = publish_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /workflow/updatestatus HTTP/1.1" 417 55


In [169]:
req_pub5.text

'The remote server returned an error: (400) Bad Request.'

### Versioning

In [182]:
ver_url = "https://ddhinboundapiuat.asestg.worldbank.org/dataset/audit/list"

In [183]:
ver_js1 = {
    "data": {
        "id": "f3ba5f65-efe2-ea11-a813-000d3a8b33eb"
    }
}


In [184]:
req_ver1 = requests.post(ver_url, json = ver_js1, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /dataset/audit/list HTTP/1.1" 200 None


In [186]:
req_ver1.json()

{'dataset_id': 'f3ba5f65-efe2-ea11-a813-000d3a8b33eb',
 'audit_items': [{'id': '87deb2a5-f6e2-ea11-a813-000d3a191e87',
   'version_number': '2',
   'version_name': 'v1 publish',
   'version_id': '@2020-08-20T15:06:08.0401145Z'},
  {'id': '9162a36d-22e3-ea11-a813-000d3a191e87',
   'version_number': '5',
   'version_name': 'v6 publish',
   'version_id': '@2020-08-20T20:19:37.5821407Z'},
  {'id': '04cccd50-9ee7-ea11-a817-000d3a191e87',
   'version_number': '6',
   'version_name': 'v5 publish',
   'version_id': '@2020-08-26T13:16:29.3988111Z'},
  {'id': 'fa9dbe98-f5e2-ea11-a813-000d3a8b33eb',
   'version_number': '1',
   'version_name': 'v1 publish',
   'version_id': '@2020-08-20T14:58:35.8607903Z'},
  {'id': '6b9575b3-f7e2-ea11-a813-000d3a8b33eb',
   'version_number': '3',
   'version_name': 'v3 publish',
   'version_id': '@2020-08-20T15:13:44.6608588Z'},
  {'id': '32d0c1bd-fbe2-ea11-a813-000d3a8b33eb',
   'version_number': '4',
   'version_name': 'v5 publish',
   'version_id': '@2020-08-

In [81]:
ver_js2 = {
    "data": {
        "id": "a5e17900-f6e2-ea11-a813-000d3a8b33eb"
    }
}

req_ver2 = requests.post("https://ddhinboundapiuat.asestg.worldbank.org/resource/audit/list", json = ver_js2, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /resource/audit/list HTTP/1.1" 200 None


In [82]:
req_ver2.json()

{'dataset_resource_id': 'a5e17900-f6e2-ea11-a813-000d3a8b33eb',
 'audit_items': [{'id': 'cbdeb2a5-f6e2-ea11-a813-000d3a191e87',
   'version_number': '2',
   'version_name': 'v1 publish'},
  {'id': 'bc62a36d-22e3-ea11-a813-000d3a191e87',
   'version_number': '5',
   'version_name': 'v6 publish'},
  {'id': '289b72b9-f7e2-ea11-a813-000d3a8b33eb',
   'version_number': '3',
   'version_name': 'v3 publish'},
  {'id': '98d0c1bd-fbe2-ea11-a813-000d3a8b33eb',
   'version_number': '4',
   'version_name': 'v5 publish'}]}

#### View metadata versions

In [46]:
vver_url = "https://ddhinboundapiuat.asestg.worldbank.org/dataset/audit/view"

In [47]:
vver_js1 = {
    "data" : {
        "id" : "fa9dbe98-f5e2-ea11-a813-000d3a8b33eb"
    }
}

req_vv1 = requests.post(vver_url, json = vver_js1, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /dataset/audit/view HTTP/1.1" 200 None


In [48]:
req_vv1.json()

{'id': 'fa9dbe98-f5e2-ea11-a813-000d3a8b33eb',
 'name': 'Workflow Test 1',
 'unique_id': '0001103',
 'archive_nid': None,
 'status': 'PUBLISHED',
 'identification': {'id': '169ebe98-f5e2-ea11-a813-000d3a8b33eb',
  'title': 'Workflow Test 1',
  'subtitle': 'testing the basic workflow',
  'description': None,
  'acronym': 'WDI',
  'topics': None,
  'practice': {'code': 'POV', 'name': 'Poverty and Equity'},
  'work_unit': None,
  'working_unit_vpu': None,
  'wb_project_reference': None,
  'grant_number': 'Grant number 01',
  'parent_dataset': None,
  'collection_id': None,
  'translated_title': 'World Development Indicators TEST ENG',
  'point_of_contact': [{'id': '269ebe98-f5e2-ea11-a813-000d3a8b33eb',
    'name': 'Gaurav Bhardwaj',
    'role': {'code': 'OWNER', 'name': 'Owner'},
    'email': 'gbhardwaj@worldbank.org',
    'type': 'TTL',
    'upi': '542830',
    'is_emailaddress_visibility_externally': True},
   {'id': '279ebe98-f5e2-ea11-a813-000d3a8b33eb',
    'name': 'User 2',
    'ro

In [49]:
vver_js2 = {
    "data" : {
        "id" : "32d0c1bd-fbe2-ea11-a813-000d3a8b33eb"
    }
}

req_vv2 = requests.post(vver_url, json = vver_js2, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /dataset/audit/view HTTP/1.1" 200 None


In [50]:
req_vv2.json()

{'id': '32d0c1bd-fbe2-ea11-a813-000d3a8b33eb',
 'name': 'Workflow Test 1',
 'unique_id': '0001103',
 'archive_nid': None,
 'status': 'PUBLISHED',
 'identification': {'id': '71d0c1bd-fbe2-ea11-a813-000d3a8b33eb',
  'title': 'Workflow Test 1',
  'subtitle': 'testing the basic workflow',
  'description': None,
  'acronym': 'WDI',
  'topics': None,
  'practice': {'code': 'POV', 'name': 'Poverty and Equity'},
  'work_unit': None,
  'working_unit_vpu': None,
  'wb_project_reference': None,
  'grant_number': 'Grant number 01',
  'parent_dataset': None,
  'collection_id': None,
  'translated_title': 'World Development Indicators TEST ENG',
  'point_of_contact': [{'id': '8cd0c1bd-fbe2-ea11-a813-000d3a8b33eb',
    'name': 'Gaurav Bhardwaj',
    'role': {'code': 'OWNER', 'name': 'Owner'},
    'email': 'gbhardwaj@worldbank.org',
    'type': 'TTL',
    'upi': '542830',
    'is_emailaddress_visibility_externally': True},
   {'id': '8ed0c1bd-fbe2-ea11-a813-000d3a8b33eb',
    'name': 'User 2',
    'ro

In [83]:
vver_js3 = {
    "data" : {
        "id" : "bc62a36d-22e3-ea11-a813-000d3a191e87"
    }
}

req_vv3 = requests.post("https://ddhinboundapiuat.asestg.worldbank.org/resource/audit/view", json = vver_js3, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /resource/audit/view HTTP/1.1" 200 None


In [84]:
req_vv3.json()

{'id': 'bc62a36d-22e3-ea11-a813-000d3a191e87',
 'name': 'Workflow Test - Resource 2',
 'unique_id': 'DR0001056',
 'archive_nid': None,
 'dataset': {'id': 'f3ba5f65-efe2-ea11-a813-000d3a8b33eb',
  'name': 'Workflow Test 1'},
 'identification': {'id': 'c662a36d-22e3-ea11-a813-000d3a191e87',
  'name': 'Workflow Test - Resource 2',
  'type': None,
  'description': None,
  'preview_image_url': None,
  'status': None,
  'status_log': None,
  'work_unit': None,
  'file_name': None,
  'version': None,
  'version_date': None,
  'citation': None,
  'tags': None},
 'constraints': {'id': 'bf62a36d-22e3-ea11-a813-000d3a191e87',
  'security': {'data_classification_of_file': None,
   'exception': 'Third party confidence'},
  'license': {'license_id': None,
   'custom_license_information': None,
   'license_reference': None,
   'license_exception': 'Third party confidence'},
  'name': 'Workflow Test - Resource 2'},
 'distribution': {'id': 'c362a36d-22e3-ea11-a813-000d3a191e87',
  'format': None,
  'ge

#### Update metadata of published resource

In [73]:
##a5e17900-f6e2-ea11-a813-000d3a8b33eb

pub_res_url = "https://ddhinboundapiuat.asestg.worldbank.org/resource/update"

pub_res_js = {
      "Resource": {
        "id": "a5e17900-f6e2-ea11-a813-000d3a8b33eb",
          
          "identification": {
        "id": "a6e17900-f6e2-ea11-a813-000d3a8b33eb",
        "name": "Workflow Test - Resource 2"},
          
          "geographical_extent": {"id": "a9e17900-f6e2-ea11-a813-000d3a8b33eb",
            #"geo_coverage": "WBG geo_coverage",
            "countries": [
                {
                    "code": "IN"
                }
            ],
            "geo_json": str(geo_js),
            "bounding_box": {
                "west_longitude": "77.1240234375",
                "east_longitude": "28.256005619824972",
                "north_latitude": "77.684326171875",
                "south_latitude": "28.8831596093235"
            }
        }
          
      }
}

req_res_pub = requests.post(pub_res_url, json = pub_res_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /resource/update HTTP/1.1" 200 None


In [74]:
req_res_pub.text

'EntityID: a5e17900-f6e2-ea11-a813-000d3a8b33eb'

#### Uploading data via requests

In [137]:
files = {'file': open(r'test_doc.xlsx', 'rb')}

upload_url = "https://ddhinboundapiuat.asestg.worldbank.org/resource/upload"

upload_js = {
"DatasetUniqueNumber":"0001103",
"DataSetId":"f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
"DataResourceUniqueNumber":"DR0001056",
"ResourceId":"a5e17900-f6e2-ea11-a813-000d3a8b33eb",
"FileName":"test_doc.xlsx",
"MimeType":"xlsx"
}
dat = {"param" : upload_js}
req_upload = requests.post(upload_url, json = dat, files=files, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /resource/upload HTTP/1.1" 200 188


In [138]:
req_upload.text

'{"Success":false,"Message":null,"FileUrl":null,"FileType":null,"FileSize":null,"DataSetId":null,"ResourceId":null}'

In [ ]:
'content-type': "multipart/form-data",
'Content-Type': "application/x-www-form-urlencoded",

In [118]:
dataset.ddh_header.update({"content-type": "multipart/form-data"}) #"Content-Type": "application/x-www-form-urlencoded"

### Indicators

In [139]:
ind_js = {
    "Indicator": {
        "dataset_id": "f3ba5f65-efe2-ea11-a813-000d3a8b33eb",
        "fields": [
            {
                "name": "Workflow test Indicator",
                "title": "Some title for indicator",
                "type": "POV related",
                #"format": "A string specifying a format",
                #"rdfType": "URI of RDF class containing semantic description of field",
                "description": "A description for the field",
                "constraints": "a constraints-descriptors",
                "unit": "Unit of measurement",
                "unitSystem": "reference to system of units"
            }
        ]
    }
}


In [142]:
ind_cr_url = "https://ddhinboundapiuat.asestg.worldbank.org/indicator/create"

req_ind = requests.post(ind_cr_url, json = ind_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /indicator/create HTTP/1.1" 200 None


In [143]:
req_ind.text

'EntityID: 72c76eb4-c6e3-ea11-a813-000d3a8b33eb'

In [157]:
dat_view= "https://ddhinboundapiuat.asestg.worldbank.org/dataset/view"

view_js = {
    "data": {
        "id" : "f3ba5f65-efe2-ea11-a813-000d3a8b33eb"
    }
}

req_ind = requests.post(dat_view, json = view_js, headers = dataset.ddh_header)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ddhinboundapiuat.asestg.worldbank.org:443
DEBUG:urllib3.connectionpool:https://ddhinboundapiuat.asestg.worldbank.org:443 "POST /dataset/view HTTP/1.1" 200 None
